In [1]:
import numpy as np
import pandas as pd

import re
import string 
import contractions 

from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

/Users/zankhanamehta/Documents/NEU/NLP/Parse-Resume-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading JD Data

In [2]:
jd_data = load_dataset('jacob-hugging-face/job-descriptions', split="train")
jd_data

Dataset({
    features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
    num_rows: 853
})

In [3]:
jd_df = pd.DataFrame(jd_data)
jd_df.head()

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."


In [4]:
# Sample JD

# jd_df['model_response'][0]
print(jd_df['job_description'][0])

minimum qualifications
bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles
preferred qualifications
 years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills
about the job
as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organizations more product

## Loading Extracted CV Data

In [5]:
df = pd.read_csv('data/extracted_resume.csv')
df.head()

,Skills,Education,ID,Category
0,Excellent classroom managementÂ,Subject Matter Authorization in Science: Scien...,37201447,AGRICULTURE
1,"Team mediation, Budget Management, Delegation ...","2009 Howard University ï¼​ City , State , USA ...",12674256,AGRICULTURE
2,"COMPUTER LITERACY, E-mail, English, government...","2011\nThe Universty of Zambia ï¼​ City , State...",29968330,AGRICULTURE
3,"C, C++, communication skills, designing, ELISA...","Masters of Science , Biotechnology 5 2013 Univ...",81042872,AGRICULTURE
4,"Data Entry, Printers, Clients, Loans, Tax Retu...",Wayne State University 2013 MBA : Linguistics ...,20006992,AGRICULTURE


In [6]:
df.shape

(2484, 4)

## Basic text cleaning

And doing some preprocessing in the data as discussed in the EDA notebook

In [7]:
def text_cleaning(text:str) -> str:
    if pd.isnull(text):
        return
    
    # lower-case everything
    text = text.lower().strip()
    
    # For removing puctuations
    translator = str.maketrans('', '', string.punctuation)
    
    # expand all the short-form words
    text = contractions.fix(text)
    
    # remove any special chars
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) # Remove URLs
    text = re.sub(r'\S+@\S+', '', text) # Remove emails
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text) # Remove phone numbers
    text = text.translate(translator) # Remove puctuations
    text = re.sub(r'[^a-zA-Z]', ' ', text) # Remove other non-alphanumeric characters
    
    return text.strip()

In [8]:
# We have 15 Resumes where Skills & Education were not extracted -- REFER EDA notebook
# So, let's remove them
cv_df = df[~(df['Skills'].isna() & df['Education'].isna())].reset_index(drop=True)

# Filling the null values in Skills & Education with Empty String before concatinating them
cv_df = cv_df.fillna(value='')

# Let's stitch together Skills & Education, similar to given in job description.
cv_df['CV'] = cv_df['Skills'] + ' ' + cv_df['Education']

# Doing text cleaning
cv_df['CV'] = cv_df['CV'].progress_apply(text_cleaning)

Progress Bar: 100%|██████████| 2469/2469 [00:00<00:00, 18340.36it/s]


In [9]:
cv_df.shape

(2469, 5)

In [10]:
# Sample job descriptions
job_descriptions = jd_df['job_description'].apply(text_cleaning)[:15].to_list() # jd_df['job_description'][:15]

# Sample resumes (replace with your extracted resume data)
resumes = cv_df['CV'].to_list()

## Creating Embedding using `DistilBertTokenizer`, `DistilBertModel`

In [11]:
%%time

# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


# Tokenize and embed job descriptions
job_description_embeddings = []
for description in job_descriptions:
    tokens = tokenizer(description, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).numpy()
    job_description_embeddings.append(embeddings[0])  # Flatten the embeddings to 1D

# Tokenize and embed resumes
resume_embeddings = []
for resume in resumes:
    tokens = tokenizer(resume, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).numpy()
    resume_embeddings.append(embeddings[0])  # Flatten the embeddings to 1D

/Users/zankhanamehta/Documents/NEU/NLP/Parse-Resume-main/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 53.5 s, sys: 22.6 s, total: 1min 16s
Wall time: 55.9 s


In [12]:
job_description_embeddings[0].shape, resume_embeddings[0].shape

((768,), (768,))

In [13]:
len(job_description_embeddings), len(resume_embeddings)

(15, 2469)

## Calculating Similarity Score & Getting Top 5 Candidates

In [14]:
# Calculate cosine similarity between job descriptions and resumes
similarity_scores = cosine_similarity(job_description_embeddings, resume_embeddings)
similarity_scores

array([[0.78798974, 0.8061505 , 0.78997135, ..., 0.72221214, 0.7781182 ,
        0.6956936 ],
       [0.7847351 , 0.8249048 , 0.7690108 , ..., 0.702317  , 0.77627593,
        0.70129955],
       [0.77788216, 0.78859985, 0.79538816, ..., 0.7024075 , 0.7763267 ,
        0.7020688 ],
       ...,
       [0.80130345, 0.7965963 , 0.7934847 , ..., 0.7281854 , 0.7757018 ,
        0.71121   ],
       [0.8347737 , 0.8271105 , 0.8366688 , ..., 0.8582534 , 0.8394397 ,
        0.7662347 ],
       [0.81139696, 0.80865467, 0.819006  , ..., 0.74138117, 0.78684676,
        0.7175248 ]], dtype=float32)

In [15]:
# Rank candidates for each job description based on similarity scores
num_top_candidates = 10
top_candidates = []

for i, job_description in enumerate(job_description_embeddings):
    candidates_with_scores = list(enumerate(similarity_scores[i]))
    candidates_with_scores.sort(key=lambda x: x[1], reverse=True)
    top_candidates_for_job = candidates_with_scores[:num_top_candidates]
    top_candidates.append(top_candidates_for_job)

# Print the top candidates for each job description
for i, job_description in enumerate(job_descriptions):
    print(f"Top candidates for JD {i+1} - Postition: {jd_df['position_title'][i]}")
    for candidate_index, score in top_candidates[i]:
        print(f"  Candidate {candidate_index + 1} - Similarity Score: {score:.4f} - {cv_df['Category'][candidate_index]}/{cv_df['ID'][candidate_index]}.pdf")
        # print(f"  Resume: {resumes[candidate_index]}")
    print()

Top candidates for JD 1 - Postition: Sales Specialist
  Candidate 1374 - Similarity Score: 0.9415 - HR/18827609.pdf
  Candidate 50 - Similarity Score: 0.9388 - AGRICULTURE/62994611.pdf
  Candidate 125 - Similarity Score: 0.9387 - ARTS/43622023.pdf
  Candidate 1160 - Similarity Score: 0.9377 - ACCOUNTANT/16237710.pdf
  Candidate 661 - Similarity Score: 0.9314 - HEALTHCARE/10466208.pdf
  Candidate 1358 - Similarity Score: 0.9307 - HR/27165830.pdf
  Candidate 754 - Similarity Score: 0.9303 - PUBLIC-RELATIONS/12237267.pdf
  Candidate 73 - Similarity Score: 0.9269 - ARTS/12413512.pdf
  Candidate 1402 - Similarity Score: 0.9258 - HR/21830565.pdf
  Candidate 2169 - Similarity Score: 0.9258 - ADVOCATE/18090899.pdf

Top candidates for JD 2 - Postition: Apple Solutions Consultant
  Candidate 2112 - Similarity Score: 0.9236 - ADVOCATE/22391901.pdf
  Candidate 1944 - Similarity Score: 0.9165 - BUSINESS-DEVELOPMENT/95382114.pdf
  Candidate 1799 - Similarity Score: 0.9159 - FITNESS/21238396.pdf
  Ca

In [16]:
selected_jd_indices = [0, 1, 2, 3, 4]

top_data = []

for jd_index in selected_jd_indices:
    # Top matching resume for this JD
    top_resume_index, top_score = top_candidates[jd_index][0]
    
    # Extract resume details
    resume_id = cv_df['ID'][top_resume_index]
    resume_cat = cv_df['Category'][top_resume_index]
    
    top_data.append({
        "JD Title": jd_df['position_title'][jd_index],
        "Resume ID": resume_id,
        "Resume Category": resume_cat,
        "Similarity Score": top_score
    })

top_resumes_df = pd.DataFrame(top_data)
top_resumes_df
# print(top_resumes_df)


,JD Title,Resume ID,Resume Category,Similarity Score
0,Sales Specialist,18827609,HR,0.941542
1,Apple Solutions Consultant,22391901,ADVOCATE,0.923647
2,Licensing Coordinator - Consumer Products,43622023,ARTS,0.959810
3,Web Designer,18979238,DESIGNER,0.948679
4,Web Developer,62994611,AGRICULTURE,0.950087


In [17]:
avg_top10_scores = []

for jd_index in selected_jd_indices:
    top_10 = top_candidates[jd_index][:10]
    
    scores = [score for _, score in top_10]
    
    avg_score = sum(scores) / len(scores)
    
    avg_top10_scores.append({
        "JD Title": jd_df['position_title'][jd_index],
        "Average Top-10 Score": round(avg_score, 4)
    })

avg_top10_df = pd.DataFrame(avg_top10_scores)

print(avg_top10_df)

                                    JD Title  Average Top-10 Score
0                           Sales Specialist                0.9328
1                 Apple Solutions Consultant                0.9118
2  Licensing Coordinator - Consumer Products                0.9444
3                               Web Designer                0.9360
4                              Web Developer                0.9395


In [ ]:
# Fixed selections
selected_jd_indices = [0, 1, 2, 3, 4]
selected_resume_indices = [1160, 1799, 2112, 1268, 1430, 1360, 713, 1944, 663, 1551]

# Initialize matrix
distilbert_matrix = []

for resume_idx in selected_resume_indices:
    row = []
    for jd_idx in selected_jd_indices:
        sim_score = similarity_scores[jd_idx][resume_idx]
        row.append(round(sim_score, 4))
    distilbert_matrix.append(row)

# Prepare column & row labels
jd_titles = [jd_df["position_title"][i] for i in selected_jd_indices]
resume_labels = [f"{cv_df['Category'][i]}/{cv_df['ID'][i]}" for i in selected_resume_indices]

# Create DataFrame
distilbert_df = pd.DataFrame(distilbert_matrix, index=resume_labels, columns=jd_titles)
distilbert_df


,Sales Specialist,Apple Solutions Consultant,Licensing Coordinator - Consumer Products,Web Designer,Web Developer
ACCOUNTANT/75286906,0.8135,0.7728,0.8022,0.8393,0.8198
FITNESS/27974588,0.5128,0.5518,0.4806,0.5255,0.5242
ADVOCATE/14445309,0.7893,0.7413,0.7874,0.8238,0.7958
INFORMATION-TECHNOLOGY/29975124,0.7962,0.7672,0.7652,0.8079,0.8051
CONSTRUCTION/30397268,0.7297,0.7406,0.7239,0.7714,0.7602
HR/22323967,0.7436,0.7491,0.7404,0.7734,0.7574
HEALTHCARE/24025053,0.7677,0.7383,0.7625,0.7936,0.7885
BUSINESS-DEVELOPMENT/12546838,0.4016,0.4402,0.3852,0.4247,0.4097
HEALTHCARE/23944036,0.7590,0.7608,0.7582,0.8090,0.7893
DESIGNER/25023614,0.8066,0.8309,0.7927,0.8270,0.8180
